In [1]:
from rich.console import Console
from rich_theme_manager import ThemeManager
import pathlib
import warnings

"""
This is a RAG that:
 - gets mocked up data from the APSViz DB
 - uses a mini-LLM for the NLM.
"""
from dotenv import load_dotenv
import os
import psycopg2

# load the secret DB credentials
load_dotenv()

# load the theme
theme_dir = pathlib.Path("themes")
theme_manager = ThemeManager(theme_dir=theme_dir)
dark = theme_manager.get("dark")

# create a console with the dark theme
console = Console(theme=dark)

# suppress warnings
warnings.filterwarnings('ignore')

device = "cpu"  # set to "cuda" if you have GPU and torch.cuda.is_available()


In [2]:
def run_query(query):
    """
    runs a query against the APSViz DB.

    Note this notebook expects localhost to be connected to a postgres DB.
    :param query:
    :return:
    """
    # Database connection parameters
    connection = psycopg2.connect(dbname="apsviz", user=os.getenv("PG_USER"), password=os.getenv("PG_USER"), host="localhost", port="5432")

    results = None

    with connection.cursor() as cursor:
        try:
            # Create a cursor object
            cursor = connection.cursor()

            # Execute an SQL query
            cursor.execute(query)

            # Fetch and print results
            results = cursor.fetchall()

        except Exception as e:
            print("An error occurred:", e)

        finally:
            # Close the cursor and connection
            cursor.close()
            connection.close()

    return results[0][0]

In [3]:
# create the SQL and get mocked up data from the DB.
# note we are converting number values to feet.
query = """
            SELECT json_agg(row_to_json(t))
            FROM (
                SELECT name, station_id, abbrev, lon, lat,
                CASE WHEN nos_minor IS NOT NULL THEN (nos_minor * 3.28084) ELSE NULL END AS nos_minor,
                CASE WHEN nos_moderate IS NOT NULL THEN (nos_moderate * 3.28084) ELSE NULL END AS nos_moderate,
                CASE WHEN nos_major IS NOT NULL THEN (nos_major * 3.28084) ELSE NULL END AS nos_major,
                CASE WHEN nws_minor IS NOT NULL THEN (nws_minor * 3.28084) ELSE NULL END AS nws_minor,
                CASE WHEN nws_moderate IS NOT NULL THEN (nws_moderate * 3.28084) ELSE NULL END AS nws_moderate,
                CASE WHEN nws_major IS NOT NULL THEN (nws_major * 3.28084) ELSE NULL END AS nws_major,
                FLOOR(random() * 5 + 1)::INT AS current_level
                FROM noaa_station_levels
                ORDER BY name
            ) t ;
        """
# get the station data
data = run_query(query)

In [4]:
def get_flood_stage(values):
    """
    Gets the flood stage based on the station data
    Note "current_data" is a random number (1 to 5) already generated in the data
    """
    if ((values['nos_major'] and values['nos_major'] - values['current_level'] < 0) or (
            values['nws_major'] and values['nws_major'] - values['current_level'] < 0)):
        return 'major flooding'
    elif ((values['nos_moderate'] and values['nos_moderate'] - values['current_level'] < 0) or (
            values['nws_moderate'] and values['nws_moderate'] - values['current_level'] < 0)):
        return 'moderate flooding'
    elif ((values['nos_minor'] and values['nos_minor'] - values['current_level'] < 0) or (
            values['nws_minor'] and values['nws_minor'] - values['current_level'] < 0)):
        return 'minor flooding'
    else:
        return 'no flooding'

In [5]:
"""
get the data in an acceptable format
"""

docs = []
metadata = []

# for eact record returned from the DB
for d in data:
    # get the flood stage label for this station
    flood_stage = get_flood_stage(d)

    # create some data tags
    tags = [str(d['station_id']), d['name'], flood_stage]

    # get the data in a understandable format
    text = f"{d['name']}: {flood_stage} (tags: {','.join(tags)})"

    # save the
    docs.append(text)
    metadata.append(d)

In [6]:
"""
embed that data and create an embedding index
"""
from sentence_transformers import SentenceTransformer
import faiss

# load the transformer
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# encode the data
doc_embeddings = embedder.encode(docs, convert_to_numpy=True)

# get the extents of the data
dim = doc_embeddings.shape[1]

# create an index
index = faiss.IndexFlatL2(dim)

# add the embeddings to the index
index.add(doc_embeddings)

In [7]:
def retrieve_with_scores(query, top_k=2):
    """
    gets the answer data with the scores

    :param query:
    :param top_k:
    :return:
    """
    # encode the query
    q_emb = embedder.encode([query], convert_to_numpy=True)  # (1, dim)

    # get the scores and answer/data indexes
    scores, idx = index.search(q_emb, top_k)  # returned L2 dists

    # convert L2 -> similarity by negative distance (simple)
    similarity = -scores[0].astype(float)  # length top_k

    # init the results
    results = []

    # loop through the answers
    for i in idx[0]:
        # put away the result
        results.append({"text": docs[i], "metadata": metadata[i]})

    # get the embeddings of the answers
    retrieved_embeddings = doc_embeddings[idx[0]]

    return results, similarity, retrieved_embeddings

In [8]:
results, similarity, retrieved_embeddings = retrieve_with_scores("places with minor flooding", 2)

print('results:', results, 'similarity:', similarity)  # , 'embeddings:', retrieved_embeddings

results: [{'text': 'Cambridge: moderate flooding (tags: 8571892,Cambridge,moderate flooding)', 'metadata': {'name': 'Cambridge', 'station_id': 8571892, 'abbrev': 'Cambridg', 'lon': -76.061667, 'lat': 38.5725, 'nos_minor': 2.7407017073170734, 'nos_moderate': 3.71095012195122, 'nos_major': 4.94126512195122, 'nws_minor': 2.4806351219512197, 'nws_moderate': 2.980763170731708, 'nws_major': 3.4808912195121953, 'current_level': 3}}, {'text': 'Dahlgren: minor flooding (tags: 8635027,Dahlgren,minor flooding)', 'metadata': {'name': 'Dahlgren', 'station_id': 8635027, 'abbrev': 'Dahlgren', 'lon': -77.036597, 'lat': 38.319753, 'nos_minor': None, 'nos_moderate': None, 'nos_major': None, 'nws_minor': 2.5606556097561, 'nws_moderate': 4.061039756097562, 'nws_major': 6.071554512195122, 'current_level': 4}}] similarity: [-0.63787949 -0.74785548]


In [9]:
from transformers import RagSequenceForGeneration, DPRQuestionEncoderTokenizer, DPRContextEncoderTokenizer, BartTokenizer

model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq").to(device)
q_tok = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
ctx_tok = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
gen_tok = BartTokenizer.from_pretrained("facebook/bart-large")

Some weights of the model checkpoint at facebook/rag-sequence-nq were not used when initializing RagSequenceForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagSequenceForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagSequenceForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class

In [10]:
def fallback_concat_generate(query: str, contexts):
    """
    dumps out the data when there has been an exception

    :param query:
    :param contexts:
    :return:
    """
    # create the model with the same generator weights
    gen_model = model.generator.to(device)

    # Build prompt: query + contexts joined (keeps provenance)
    full_prompt = query + " " + " ".join([f"[DOC{i}] {c}" for i, c in enumerate(contexts, start=1)])

    # get the input tokens
    inputs = gen_tok(full_prompt, return_tensors="pt", truncation=True, padding=True).to(device)

    # generate the output
    out = gen_model.generate(input_ids=inputs["input_ids"],
                             attention_mask=inputs["attention_mask"],
                             max_length=64,
                             num_beams=2)

    # get the answer
    answer = gen_tok.batch_decode(out, skip_special_tokens=True)[0]

    # return the results
    return {"query": query, "answer": answer, "sources": contexts, "note": "fallback_concat_used"}

In [11]:
import torch

def rag_answer(query, top_k=2):
    """
    gets the answer

    :param query:
    :param top_k:
    :return:
    """
    # get the results
    retrieved_texts, sims, retrieved_embs = retrieve_with_scores(query, top_k=top_k)

    # get a list of the text outputs
    texts = [r["text"] for r in retrieved_texts]

    # get the length of the results
    k = len(texts)

    # make sure we have results
    if k == 0:
        return {"query": query, "answer": "(no answers)", "sources": []}

    # tell the model how many docs per query
    model.config.n_docs = k

    # tokenize the question (DPRQuestion tokenizer)
    q_inputs = q_tok(query, return_tensors="pt", truncation=True, padding=True).to(device)

    # tokenize the contexts (DPRContext tokenizer)
    ctx_inputs = ctx_tok(texts, return_tensors="pt", truncation=True, padding=True).to(device)

    # build doc_scores tensor: shape (batch_size, n_docs) => here batch_size=1
    # Use similarities computed from FAISS (converted to float). Make sure same device and dtype as
    # else
    doc_scores = torch.tensor(sims, dtype=torch.float32, device=device).unsqueeze(0)  # (1, k)

    # debug prints to understand the results
    # print("DEBUG SHAPES:")
    # print(" q input_ids:", q_inputs["input_ids"].shape, "dtype:", q_inputs["input_ids"].dtype)
    # print(" q attention_mask:", q_inputs["attention_mask"].shape)
    # print(" ctx input_ids:", ctx_inputs["input_ids"].shape, "dtype:", ctx_inputs["input_ids"].dtype)
    # print(" ctx attention_mask:", ctx_inputs["attention_mask"].shape)
    # print(" model.config.n_docs:", model.config.n_docs)
    print(" doc_scores:", doc_scores.shape, "values:", doc_scores.tolist())

    # final safety checks
    assert "input_ids" in q_inputs and "input_ids" in ctx_inputs, "missing tokenized ids"
    assert q_inputs["input_ids"].dtype == torch.long and ctx_inputs["input_ids"].dtype == torch.long
    assert doc_scores.shape[1] == ctx_inputs["input_ids"].shape[0], f"doc_scores dim mismatch: {doc_scores.shape[1]} vs ctx count {ctx_inputs['input_ids'].shape[0]}"

    try:
        # TODO: why doesnt this work?
        #  get the RAG results
        out_ids = model.generate(input_ids=q_inputs["input_ids"],
                                 attention_mask=q_inputs["attention_mask"],
                                 context_input_ids=ctx_inputs["input_ids"],
                                 context_attention_mask=ctx_inputs["attention_mask"],
                                 doc_scores=doc_scores, n_docs=k, max_length=64, num_beams=2, )

        # grab the answer
        answer = gen_tok.batch_decode(out_ids, skip_special_tokens=True)[0]

        return {"query": query, "answer": answer, "sources": texts}

    except AssertionError as e:
        console.print("RAG generate assertion ERROR:", e)

        # fall back to concatenation method (guaranteed to work)
        return fallback_concat_generate(query, texts)
    except Exception as e:
        console.print("RAG generate other ERROR:", type(e).__name__, str(e))

        # fall back to concatenation method (guaranteed to work)
        return fallback_concat_generate(query, texts)

In [12]:
prompts = ['places that have major flooding', 'places that have moderate flooding', 'places that have minor flooding', 'places that have no flooding']

[print(rag_answer(x, 2)) for x in prompts]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


 doc_scores: torch.Size([1, 2]) values: [[-0.7302360534667969, -0.7324217557907104]]


RAG generate assertion ERROR: Make sure that `context_input_ids` are passed, if no `retriever` is set. 
Alternatively, you can set a retriever using the `set_retriever(...)` function.

{'query': 'places that have major flooding', 'answer': ' 8724580', 'sources': ['Cambridge: moderate flooding (tags: 8571892,Cambridge,moderate flooding)', 'Key West: major flooding (tags: 8724580,Key West,major flooding)'], 'note': 'fallback_concat_used'}
 doc_scores: torch.Size([1, 2]) values: [[-0.5711202621459961, -0.6826287508010864]]


RAG generate assertion ERROR: Make sure that `context_input_ids` are passed, if no `retriever` is set. 
Alternatively, you can set a retriever using the `set_retriever(...)` function.

...

RAG generate assertion ERROR: Make sure that `context_input_ids` are passed, if no `retriever` is set. 
Alternatively, you can set a retriever using the `set_retriever(...)` function.

{'query': 'places that have minor flooding', 'answer': ' newport', 'sources': ['Cambridge: moderate flooding (tags: 8571892,Cambridge,moderate flooding)', 'Newport: moderate flooding (tags: 8452660,Newport,moderate flooding)'], 'note': 'fallback_concat_used'}
 doc_scores: torch.Size([1, 2]) values: [[-0.8096615076065063, -0.8138640522956848]]


RAG generate assertion ERROR: Make sure that `context_input_ids` are passed, if no `retriever` is set. 
Alternatively, you can set a retriever using the `set_retriever(...)` function.

{'query': 'places that have no flooding', 'answer': ' 8461490', 'sources': ['Matagorda City: no flooding (tags: 8773146,Matagorda City,no flooding)', 'New London: no flooding (tags: 8461490,New London,no flooding)'], 'note': 'fallback_concat_used'}


[None, None, None, None]